In [19]:
from gensim.summarization import summarize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from gensim.models import Word2Vec
import skipthoughts
import os

import theano
import theano.tensor as tensor

import pickle as pkl
import numpy as np
import copy
import nltk

from collections import OrderedDict, defaultdict
from scipy.linalg import norm
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

[nltk_data] Downloading package punkt to /Users/elli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/elli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def tokenize_text(text):
    """
    Splits the text into individual sentences
    """
    sentences = nltk.sent_tokenize(text)
    return sentences   
        
def skipthought_encode(sentences):
    """
    Obtains sentence embeddings for each sentence in the text
    """
    print('Loading pre-trained models...')
    model = skipthoughts.load_model()
    encoder = skipthoughts.Encoder(model)
    print('Encoding sentences...')
    encoded =  encoder.encode(sentences)
    return encoded
        
    
def summarize(text):
    """
    Performs summarization of text
    """
    summary = []
    print('Splitting into sentences...')
    token_text = tokenize_text(text)
    print('Starting to encode...')
    enc_text = skipthought_encode(token_text)
    print('Encoding Finished')
    n_clusters = int(np.ceil(len(enc_text)*0.3))
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans = kmeans.fit(enc_text)
    avg = []
    closest = []
    for j in range(n_clusters):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,\
                                                   enc_text)
    ordering = sorted(range(n_clusters), key=lambda k: avg[k])
    summary = ' '.join([token_text[closest[idx]] for idx in ordering])
    print('Clustering Finished')
    return summary

In [25]:
text = "After the unsuccessful attempt by Luna 1 to land on the Moon in 1959, the Soviet Union performed the first hard Moon landing - hard meaning the spacecraft intentionally crashes into the Moon – later that same year with the Luna 2 spacecraft, a feat the U.S. duplicated in 1962 with Ranger 4. Since then, twelve Soviet and U.S. spacecraft have used braking rockets (retrorockets) to make soft landings and perform scientific operations on the lunar surface, between 1966 and 1976. In 1966 the USSR accomplished the first soft landings and took the first pictures from the lunar surface during the Luna 9 and Luna 13 missions. The U.S. followed with five uncrewed Surveyor soft landings. The Soviet Union achieved the first uncrewed lunar soil sample return with the Luna 16 probe on 24 September 1970. This was followed by Luna 20 and Luna 24 in 1972 and 1976, respectively. Following the failure at launch in 1969 of the first Lunokhod, Luna E-8 No.201, the Luna 17 and Luna 21 were successful uncrewed lunar rover missions in 1970 and 1973. Many missions were failures at launch. In addition, several uncrewed landing missions achieved the Lunar surface but were unsuccessful, including: Luna 15, Luna 18, and Luna 23 all crashed on landing; and the U.S. Surveyor 4 lost all radio contact only moments before its landing. More recently, other nations have crashed spacecraft on the surface of the Moon at speeds of around 8,000 kilometres per hour (5,000 mph), often at precise, planned locations. These have generally been end-of-life lunar orbiters that, because of system degradations, could no longer overcome perturbations from lunar mass concentrations (masscons) to maintain their orbit. Japan's lunar orbiter Hiten impacted the Moon's surface on 10 April 1993. The European Space Agency performed a controlled crash impact with their orbiter SMART-1 on 3 September 2006. Indian Space Research Organisation (ISRO) performed a controlled crash impact with its Moon Impact Probe (MIP) on 14 November 2008. The MIP was an ejected probe from the Indian Chandrayaan-1 lunar orbiter and performed remote sensing experiments during its descent to the lunar surface.  The Chinese lunar orbiter Change 1 executed a controlled crash onto the surface of the Moon on 1 March 2009. The rover mission Change 3 soft-landed on 14 December 2013, as did its successor, Change 4, on 3 January 2019. All crewed and uncrewed soft landings had taken place on the near side of the Moon, until 3 January 2019 when the Chinese Change 4 spacecraft made the first landing on the far side of the Moon.[5] On 22 February 2019, Israeli private space agency SpaceIL launched spacecraft Beresheet from Cape Canaveral, Florida with the intention of achieving a soft landing. SpaceIL lost contact with the spacecraft and it crashed into the surface on 11 April.[6] Crewed landings Further information: Apollo program The view through the window of the Lunar Module Orion shortly after Apollo 16s landing. A total of twelve men have landed on the Moon. This was accomplished with two US pilot-astronauts flying a Lunar Module on each of six NASA missions across a 41-month period starting 20 July 1969 UTC, with Neil Armstrong and Buzz Aldrin on Apollo 11, and ending on 14 December 1972 UTC with Gene Cernan and Jack Schmitt on Apollo 17. Cernan was the last to step off the lunar surface. All Apollo lunar missions had a third crew member who remained on board the Command Module. The last three missions included a drivable lunar rover, the Lunar Roving Vehicle, for increased mobility. Scientific background To get to the Moon, a spacecraft must first leave Earths gravity well; currently, the only practical means is a rocket. Unlike airborne vehicles such as balloons and jets, a rocket can continue accelerating in the vacuum outside the atmosphere. Upon approach of the target moon, a spacecraft will be drawn ever closer to its surface at increasing speeds due to gravity. In order to land intact it must decelerate to less than about 160 kilometres per hour (99 mph) and be ruggedized to withstand a hard landing impact, or it must decelerate to negligible speed at contact for a soft landing (the only option for humans). The first three attempts by the U.S. to perform a successful hard Moon landing with a ruggedized seismometer package in 1962 all failed.[7] The Soviets first achieved the milestone of a hard lunar landing with a ruggedized camera in 1966, followed only months later by the first uncrewed soft lunar landing by the U.S. The speed of a crash landing on its surface is typically between 70 and 100 of the escape velocity of the target moon, and thus this is the total velocity which must be shed from the target moons gravitational attraction for a soft landing to occur. For Earths Moon, the escape velocity is 2.38 kilometres per second (1.48 mi/s).[8] The change in velocity (referred to as a delta-v) is usually provided by a landing rocket, which must be carried into space by the original launch vehicle as part of the overall spacecraft. An exception is the soft moon landing on Titan carried out by the Huygens probe in 2005. As the moon with the thickest atmosphere, landings on Titan may be accomplished by using atmospheric entry techniques that are generally lighter in weight than a rocket with equivalent capability. The Soviets succeeded in making the first crash landing on the Moon in 1959.[9] Crash landings[10] may occur because of malfunctions in a spacecraft, or they can be deliberately arranged for vehicles which do not have an onboard landing rocket. There have been many such Moon crashes, often with their flight path controlled to impact at precise locations on the lunar surface. For example, during the Apollo program the S-IVB third stage of the Saturn V rocket as well as the spent ascent stage of the Lunar Module were deliberately crashed on the Moon several times to provide impacts registering as a moonquake on seismometers that had been left on the lunar surface. Such crashes were instrumental in mapping the internal structure of the Moon. To return to Earth, the escape velocity of the Moon must be overcome for the spacecraft to escape the gravity well of the Moon. Rockets must be used to leave the Moon and return to space. Upon reaching Earth, atmospheric entry techniques are used to absorb the kinetic energy of a returning spacecraft and reduce its speed for safe landing.[citation needed] These functions greatly complicate a moon landing mission and lead to many additional operational considerations. Any moon departure rocket must first be carried to the Moons surface by a moon landing rocket, increasing the latters required size. The Moon departure rocket, larger moon landing rocket and any Earth atmosphere entry equipment such as heat shields and parachutes must in turn be lifted by the original launch vehicle, greatly increasing its size by a significant and almost prohibitive degree."

In [26]:
summarize(text)

Splitting into sentences...
Starting to encode...
Loading pre-trained models...
Loading model parameters...
Compiling encoders...


/Users/elli/anaconda3/lib/python3.7/site-packages/theano/scan_module/scan_perform_ext.py:76: UserWarning: The file scan_perform.c is not available. This donot happen normally. You are probably in a strangesetup. This mean Theano can not use the cython code for scan. If youwant to remove this warning, use the Theano flag'cxx=' (set to an empty string) to disable all ccode generation.
  "The file scan_perform.c is not available. This do"
/Users/elli/anaconda3/lib/python3.7/site-packages/theano/scan_module/scan_perform_ext.py:76: UserWarning: The file scan_perform.c is not available. This donot happen normally. You are probably in a strangesetup. This mean Theano can not use the cython code for scan. If youwant to remove this warning, use the Theano flag'cxx=' (set to an empty string) to disable all ccode generation.
  "The file scan_perform.c is not available. This do"
/Users/elli/anaconda3/lib/python3.7/site-packages/theano/scan_module/scan_perform_ext.py:76: UserWarning: The file scan_

Loading tables...
Packing up...
Encoding sentences...
56
33
27
10
21
16
7
46
34
31
15
18
25
24
39
30
26
11
54
17
19
51
36
48
40
32
53
13
20
44
Encoding Finished
Clustering Finished


'This was followed by Luna 20 and Luna 24 in 1972 and 1976, respectively. After the unsuccessful attempt by Luna 1 to land on the Moon in 1959, the Soviet Union performed the first hard Moon landing - hard meaning the spacecraft intentionally crashes into the Moon – later that same year with the Luna 2 spacecraft, a feat the U.S. duplicated in 1962 with Ranger 4. The European Space Agency performed a controlled crash impact with their orbiter SMART-1 on 3 September 2006. The first three attempts by the U.S. to perform a successful hard Moon landing with a ruggedized seismometer package in 1962 all failed. Since then, twelve Soviet and U.S. spacecraft have used braking rockets (retrorockets) to make soft landings and perform scientific operations on the lunar surface, between 1966 and 1976. SpaceIL lost contact with the spacecraft and it crashed into the surface on 11 April. A total of twelve men have landed on the Moon. Such crashes were instrumental in mapping the internal structure o